# HySwash: A hybrid method for nearshore wave processes

## 1. Set BlueMath environment

In [ ]:
import os
import os.path as op
import numpy as np
import utils.plotting
import bluemath_tk.topo_bathy.profiles
from bluemath_tk.datamining.lhs import LHS
from bluemath_tk.datamining.mda import MDA
from bluemath_tk.waves.series import waves_dispersion 
from bluemath_tk.wrappers.swash.swash_wrapper import HySwashVeggyModelWrapper
from bluemath_tk.core.io import load_model
import xarray as xr

root_dir = os.getcwd()
#output_dir = "/discos/rapido/outputVeggy"
output_dir = "/lustre/geocean/DATA/hidronas1/valva/Veggy_topo_alba"
templates_dir = os.path.join(root_dir, "templates", "VeggyBig")
export_dir = op.join(root_dir, "HyVeggy_exported")

swash_model=load_model(op.join(export_dir, "swash_model.pkl"))
postprocessed_output = xr.open_dataset(op.join(output_dir, "output_postprocessed.nc"))
postprocessed_clean = xr.open_dataset(op.join(output_dir, "output_postprocessed_clean.nc"))

### Build, run and postprocess cases

In [ ]:
swash_model.load_cases()
#swash_model.build_cases()
#swash_model.run_cases_in_background(launcher="serial", num_workers=1)
#swash_model.run_cases_bulk(launcher="sbatch --array=0-50 /home/grupos/geocean/valvanuz/HySwash/SlurmChy.sh")
#swash_model.monitor_cases()
swash_model.monitor_cases(value_counts="cases")
#swash_model.monitor_cases(value_counts="simple")

swash_model=load_model(op.join(export_dir, "swash_model.pkl"))

Rerun cases

In [ ]:
bad_cases_str = ",".join([str(i) for i in bad])

## 4. Data Processing

In [ ]:
postprocessed_output = swash_model.postprocess_cases(write_output_nc=True,overwrite_output=False, overwrite_output_postprocessed=False)
#postprocessed_output = swash_model.postprocess_cases(output_vars=["Ru2","Hrms"])
#postprocessed_output = swash_model.postprocess_cases(force=True,overwrite_output=False,overwrite_output_postprocessed=False)

Load postprocessed data from NetCDF File

In [3]:
output_dir = "/lustre/geocean/DATA/hidronas1/valva/Veggy_topo_alba/"
postprocessed_output = xr.open_dataset(op.join(output_dir, "output_postprocessed.nc"))


Check if data postprocessed contain Nans

In [4]:
# 120 nodes_per_wavelength ok: 421 and bad 579
ok=[]
bad=[]
for case_num in postprocessed_output.case_num.values:
    postprocessed_case = postprocessed_output.sel(case_num=case_num)
    #for var in postprocessed_case.data_vars:
    for var in ["Hrms"]:
        if postprocessed_case[var].isnull().values.any():
            # print the variable name and the first occurrence of NaN
            nan_indices = np.where(np.isnan(postprocessed_case[var].values))
            bad.append(case_num)
        else:
            ok.append(case_num)

In [5]:
# Save bad to a pickle file
import pickle
with open(op.join(export_dir, "bad_cases.pkl"), "wb") as f:
    pickle.dump(bad, f)

Create a new dataset with outputs that do not contain Nans

In [ ]:
post_clean=postprocessed_output.copy(deep=True)
# get unique values of ok
post_clean=post_clean.sel(case_num=ok)
post_clean.to_netcdf(
    os.path.join(output_dir, "output_postprocessed_clean.nc")
)

## 5. Reconstruction: Principal Component Analysis (PCA) & Radial Basis Fucntions (RBF)

In [ ]:
# Load the PCA and RBF models
pca=load_model(
        model_path=op.join(export_dir, "pca_model.pkl"),
    )

rbf=load_model(
        model_path=op.join(export_dir, f"rbf_model.pkl"),
)

In [ ]:
from utils.plotting import show_graph_for_all_vegetations

depth = np.loadtxt(os.path.join(os.getcwd(), "templates", "depth.bot"))
show_graph_for_all_vegetations(pca=pca, rbf=rbf, depth=depth, hs=2.0, hs_l0=0.02)